In [1]:
import random
# Install required libraries
# Import required libraries
import gymnasium as gym
from gymnasium import spaces
import matplotlib.pyplot as plt
import numpy as np
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import namedtuple, deque
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "cpu"
)

print(device)
CUDA_LAUNCH_BLOCKING=1

cuda


In [2]:
class Net(nn.Module):

    def __init__(self, obs, action):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(obs, 32)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, action)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [3]:
Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN:
  # initialize values
  def __init__(self, N, env):
    # initialize environment
    self.env = env
    state, info = self.env.reset()
    # initialize replay memory to capacity N
    self.replay = ReplayMemory(N)
    self.pointer = 0
    self.policy_net = Net(len(state), self.env.action_space.n).to(device)
    self.target_net = Net(len(state), self.env.action_space.n).to(device)
    self.optimizer = optim.AdamW(self.policy_net.parameters(),amsgrad=True ) #auto learning rate



  
  # Main training function
  def train(self, episodes, epsilon, discount, action_function, greedy):
    total_reward = [0] * episodes  
    TAU = .0004
    for i in range(episodes):
      # if i % 5 == 0:
      #    self.target_net.load_state_dict(self.policy_net.state_dict())
      # initialize sequence S and preprocessed sequence o
      seq  = [None , None]
      state, info = self.env.reset()
      seq[0] = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
      done = False
      rewards = 0
      eps = epsilon ** i if not greedy else 0
      step =  0
      while not done:
        step +=1
        # Select action
        action_type = action_function(seq[0], eps)
        observation, reward, terminated, truncated, _ = self.env.step(action_type.item())
        state = torch.tensor(observation, device=device, dtype=torch.float32).unsqueeze(0)
        # Set sequence
        seq[1] = state
        if terminated:
           seq[1] = None
        rewards += reward
        reward = torch.tensor([reward], device=device)
        # store transition in replay buffer
        self.replay.push(seq[0], action_type,  seq[1], reward)
        seq[0] = state
        
        self.optimize_function(discount)

        if step%10 == 0:
          target_net_state_dict = self.target_net.state_dict()
          policy_net_state_dict = self.policy_net.state_dict()
          for key in policy_net_state_dict:
              target_net_state_dict[key] = policy_net_state_dict[key]* TAU + target_net_state_dict[key]*(1- TAU)        
          self.target_net.load_state_dict(target_net_state_dict)
        done = truncated or terminated
      # Decay epsilon after every episode
      epsilon *= epsilon
      total_reward[i] = rewards  
    return total_reward
  # Determine the action for the warehouse environment
      
  def mountain_car_action(self, state, epsilon):
    if np.random.rand() < epsilon:
        action_type = self.env.action_space.sample()
    else:
        # select max(Q)
        with torch.no_grad():
            return self.policy_net(state).max(1).indices.view(1, 1) 
    return torch.tensor([[action_type]], device=device, dtype=torch.long)
  
  def optimize_function(self, discount):
    BATCH_SIZE = 256
    if len(self.replay) < BATCH_SIZE:
        return
    else:
        transitions = self.replay.sample(BATCH_SIZE)
        batch = Transition(*zip(*transitions))
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        state_action_values = self.policy_net(state_batch).gather(1, action_batch)
        next_state_values = torch.zeros(BATCH_SIZE, device=device)
        with torch.no_grad():
            next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(1).values
        expected_state_action_values = (next_state_values * discount) + reward_batch
             
        criterion = nn.SmoothL1Loss()
        loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
  # Save the current weights
  def save(self, filename):
    with open(filename, 'wb') as file:
      pickle.dump(self.policy_net.state_dict(), file,protocol=pickle.HIGHEST_PROTOCOL)


In [5]:
#Prints the reward per epsisode graph
def reward_print(reward_per_episode, episodes, info): 
    mins = int(min(reward_per_episode)) - int(min(reward_per_episode)) * (.15)
    maxs = int(max(reward_per_episode)) + int(max(reward_per_episode)) * (.3) 
    plt.figure()
    plt.plot(reward_per_episode)
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Cumulative Reward', fontsize=20)
    plt.title(f'Cumulative Reward Per Episode ({info})', fontsize=24)
    plt.xticks([0, episodes * .2, episodes * .4, episodes * .6, episodes * .8, episodes], fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylim(ymin= mins, ymax=maxs)
    plt.xlim(xmin=0, xmax=episodes)
    plt.grid()
    plt.show()

#prints the epsilon decay graph
def ep_decay(eps, episodes):
    epsilon_values = [(eps ** i) * 1 for i in range(episodes)]
    plt.figure()
    plt.plot(epsilon_values, linewidth=4)
    plt.xlabel('Episode', fontsize=20)
    plt.ylabel('Epsilon Value', fontsize=20)
    plt.title(f"Epsilon Decay for {eps}", fontsize=24)
    plt.xticks([0, episodes * .2, episodes * .4, episodes * .6, episodes * .8, episodes], fontsize=18)
    plt.yticks(fontsize=18)
    plt.ylim(ymin=0, ymax=1)
    plt.xlim(xmin=0, xmax=episodes)
    plt.grid()
    plt.show()


In [6]:
# N = 5000
# env = gym.make("MountainCar-v0")
# env.reset()
# MtnCar = DQN(N, env)

# episodes = 20
# epsilon = .8
# discount = .4
# action = MtnCar.mountain_car_action
# total_rewards = MtnCar.train(episodes, epsilon, discount, action, False)
# # MtnCar.save("drpreisl_angustsa_assignment2_part2_dqn_mountaincar.pickle")
# reward_print(total_rewards, episodes, "MountainCar")
# ep_decay(epsilon, episodes)
# total_rewards = MtnCar.train(6, epsilon, discount, action, True)
# reward_print(total_rewards, 5, "MountainCar")

In [7]:
N = 5000
env = gym.make("CartPole-v1")
env.reset()
cart = DQN(N, env)

episodes = 600
epsilon = .8
discount = .99
action = cart.mountain_car_action
total_rewards = cart.train(episodes, epsilon, discount, action, False)
# cart.save("drpreisl_angustsa_assignment2_part2_dqn_cartpole.pickle")
reward_print(total_rewards, episodes, "CartPole")
ep_decay(epsilon, episodes)
total_rewards = cart.train(6, epsilon, discount, action, True)
reward_print(total_rewards, 5, "CartPole")

NameError: name 'TAU' is not defined

In [ ]:
print(cart.policy_net.state_dict())
cart.save("drpreisl_angustsa_assignment2_part3_ddqn_cartpole.pickle")

OrderedDict({'layer1.weight': tensor([[ 3.5433e-01,  2.7465e-01,  7.5301e-01, -1.6675e-02],
        [-7.4364e-01, -1.3244e+00, -2.1997e+00, -9.4949e-01],
        [ 1.0396e-01, -1.6031e-01,  2.6495e-01,  5.4392e-02],
        [-2.6355e-01, -5.3193e-02, -1.0929e-01,  1.0894e-02],
        [ 2.6695e-01,  2.0126e-01, -1.1025e+00, -3.0423e-01],
        [ 6.2737e-01, -2.3406e-01, -3.8444e-01,  5.4150e-02],
        [-6.7813e-03,  8.7717e-02,  6.1036e-02, -3.8656e-02],
        [ 2.0772e-01, -1.8139e-01,  2.4778e-01, -2.3355e-01],
        [ 5.8415e-01,  2.6104e-01, -6.1396e-01, -3.4071e-01],
        [ 6.0256e-02,  4.1118e-01, -6.1339e-01,  2.0220e-01],
        [ 1.2182e-01, -1.8772e-01,  7.9857e-01, -9.8625e-03],
        [ 1.6319e-01, -2.2112e-02, -3.9875e-01, -1.0842e-01],
        [-1.0934e-01,  2.8547e-02,  3.4172e-01,  1.3674e-01],
        [ 1.7865e-01, -7.1368e-02, -4.3655e-02,  1.0616e-01],
        [-3.3848e-01, -1.3433e-01, -2.6145e+00, -5.8274e-01],
        [ 9.0321e-02, -2.1949e-01, -4.98